In [ ]:
import os
import librosa 
import librosa.core
import librosa.util
import numpy as np
import scipy as sp
import scipy.io
import soundfile as sf
import matplotlib.pyplot as plt
import pandas as pd

### Retrieve path to files 

In [ ]:
noise_path = "dataset/noise/"
noise_files = [f for f in os.listdir(noise_path)]

# here we will have the list of the files that have a quite accurate transcription

orig_db = pd.read_csv('csv/orig_librispeech_dev-clean.csv')


In [ ]:
orig_db['librispeech_folder']

### Create a dataframe for storing dataset information

In [ ]:
col = ['orig_name', 'orig_path', 'name', 'path', 'noise_type', 'SNR', 'talker', 'book','orig_transcription', 'orig_transcription_path']
db_info = pd.DataFrame(columns=col)

### Some parameters 

In [ ]:
## Parameters (eventually fix and remove constraint)

## noise files are wav 16 at 48000
nbit = 16
SNR = [0,5,10,15]

### Organization of the folder dataset
1. speech : noisy files 
    * file directory: one folder for each file of the LibriSpeech dataset; *example dir name*: <b>84-121550-0000</b>
        - noise type: one folder for each type of noise; *example dir name*: <b>train</b> <br> 
          here noisy speech files with different SNRs; *example file path*: <b>84-121550-0000/train/84-121550-0000_train_5.wav</b>
          
2. noise : noise files
3. orig : original files
4. trans : original transcriptions

### Add noise to clean files
Maybe we can avoid to have 3 nested for cycles?<br>
Probably can be useful to have a pipeline made of blocks

In [ ]:
# here in future we should read from a pandas dataframe having the paths for all the files we use from LibriSpeech

orig_librispeech_subset_path = ['/nas/public/dataset/LibriSpeech/dev-clean']
orig_librispeech_subset = ['dev-clean']

which_subset_compute = 0

orig_db_subset = orig_db[(orig_db['librispeech_folder'] == orig_librispeech_subset_path[which_subset_compute])]



for index,row in orig_db_subset.iterrows():
    
    origfile_path = row['path']
    clean, orig_samplerate = sf.read(origfile_path)
    clean = np.array(clean)
    
    transcription_path = row['transcription_path']
    with open(transcription_path) as f:
        content = f.readlines()
    
    for row in content:
        splitted = row.split(' ')
        if splitted[0] == origfile_name:
            transcription = ' '.join(splitted[1:])
            break
    
    for n in noise_files:
        noise_name = n.replace('.wav','')
        noise_samplerate, noise = scipy.io.wavfile.read(os.path.join(noise_path, n))
        noise = np.float32(noise)/ np.float32(2**(nbit-1))

        if(orig_samplerate > noise_samplerate):
            clean = librosa.resample(clean, orig_samplerate, noise_samplerate)
            samplerate = noise_samplerate
        else:
            noise = librosa.resample(noise, noise_samplerate, orig_samplerate)
            samplerate = orig_samplerate
            
        if (noise.shape[0] > clean.shape[0]):
            noise = noise[:clean.shape[0]]
        
        
        norm_factor = np.sqrt(np.mean(np.abs(clean)**2)/np.mean(np.abs(noise)**2))
        noise = noise*norm_factor
        
        
        for s in SNR:
            noise_gain = 1/(10**(s/20))
            noise_scaled = noise*noise_gain
            noisy = clean + noise_scaled
            
            
            if not os.path.exists(orig_librispeech_subset+'_'+noise_name):
                    os.makedirs(directory)
        
    if(index==10):
        break
    

In [ ]:
import IPython.display as ipd
ipd.Audio('/nas/public/dataset/LibriSpeech/dev-clean/2277/149896/2277-149896-0015.flac') 



### Pick some random file and check

In [ ]:
random_file = db_info.sample(n=1)
noise_samplerate, noise = scipy.io.wavfile.read(random_file['path'].values[0])
import IPython.display as ipd
ipd.Audio(noise, rate=noise_samplerate) # load a local WAV file

In [ ]:
random_file